In [1]:
img_rows = 28
img_cols = 28
channels = 1
img_shape = (img_rows, img_cols, channels)

print(img_shape)

(28, 28, 1)


In [8]:
from keras.layers import LeakyReLU
from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten
from keras.layers import BatchNormalization
from tensorflow.keras.layers import LeakyReLU
from keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
import numpy as np

def build_generator():

    noise_shape = (100,) #1D array of size 100 (latent vector / noise)
    
    model = Sequential()

    model.add(Dense(256, input_shape=noise_shape))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(512))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(1024))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    
    model.add(Dense(np.prod(img_shape), activation='tanh'))
    model.add(Reshape(img_shape))

    return model

In [15]:
def build_discriminator():


    model = Sequential()

    model.add(Flatten(input_shape=img_shape))
    model.add(Dense(512))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(256))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(1, activation='sigmoid'))

    return model

In [ ]:
def train(epochs, batch_size=128, save_interval=50):

    (X_train, _), (_, _) = mnist.load_data()
    train_data = (X_train.astype(np.float32) - 127.5) / 127.5
    train_data = train_data.reshape(train_data.shape[0],train_data.shape[1],train_data.shape[2],1)
    half_batch = int(batch_size / 2)
    
    for epoch in range(epochs):

        idx = np.random.randint(0, X_train.shape[0], half_batch)
        imgs = X_train[idx]
 
        noise = np.random.normal(0, 1, (half_batch, 100))
        gen_imgs = generator.predict(noise)

        d_loss_real = discriminator.train_on_batch(imgs, np.ones((half_batch, 1)))
        d_loss_fake = discriminator.train_on_batch(gen_imgs, np.zeros((half_batch, 1)))
        
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake) 

        gan_train_noise = np.random.normla(0,1,(batch_size, 100))
        gan_train_target = np.ones((batch_size, 1))

        discriminator.trainable = False

        g_loss = GNN.train_on_batch(gan_train_noise, gan_train_target)

        print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))

        if epoch % save_interval == 0:
            save_imgs(epoch)


In [ ]:
def save_imgs(epoch):
    r, c = 5, 5
    noise = np.random.normal(0, 1, (r * c, 100))
    gen_imgs = generator.predict(noise)

    fig, axs = plt.subplots(r, c)
    cnt = 0
    for i in range(r):
        for j in range(c):
            axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
            axs[i,j].axis('off')
            cnt += 1
    fig.savefig("images/mnist_%d.png" % epoch)
    plt.close()

In [ ]:
optimizer = Adam(0.0002, 0.5)  #Learning rate and momentum.
 
discriminator = build_discriminator()
discriminator.compile(loss='binary_crossentropy',optimizer=optimizer,metrics=['accuracy'])

generator = build_generator()
generator.compile(loss='binary_crossentropy', optimizer=optimizer)

noise_input = Input(shape=(100,))
img = generator(z)

discriminator.trainable = False  

discriminator_output = discriminator(generator_output)

GAN = Model(noise_input ,discriminator_output)
GAN.compile(loss='binary_crossentropy', optimizer=optimizer)

train(epochs=10000, batch_size=32, save_interval=10)

generator.save('generator_model.h5')  #Test the model on GAN4_predict...